### Загрузим датасет с Kaggle для предсказания целевой переменной с применением различных методов классификации

In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [2]:
# Данные по соревнованию от Kaggle, состоящие из обучающего и тестового набора
train = pd.read_csv('datasets/train.csv', index_col='id')
test = pd.read_csv('datasets/test.csv', index_col='id')

### Предобработка данных при помощи one-hot кодирования

In [3]:
# One-hot кодирование стобца color
object_train = train.select_dtypes(include='object')['color']
object_test = test.select_dtypes(include='object')['color']
num_train = train.select_dtypes(exclude='object')
num_test = test.select_dtypes(exclude = 'object')
one_hot_train = pd.get_dummies(object_train, dtype='float')
one_hot_test = pd.get_dummies(object_test, dtype='float')

In [4]:
# Объединение закодированных признаков
X_train = num_train.join(one_hot_train)
X_test = num_test.join(one_hot_test)
# Целевой признак
y_train = train['type']

### Теперь применим различные методы классификации для предсказания целевой переменной

#### 1) Метод k-ближайших соседей

In [5]:
params_for_knn = {'n_neighbors': range(1, 51, 2), 'algorithm': ['auto', 'ball_tree', 'kd_tree']}
KNN_model = KNeighborsClassifier()
gKNN_model = GridSearchCV(KNN_model, params_for_knn, cv=5)
gKNN_model.fit(X_train, y_train)
KNN_prediction = gKNN_model.predict(X_test) 

In [6]:
# Перевод предсказанных значений в серию pandas и сохранение в csv-файл
predict_knn = pd.Series(KNN_prediction, index=test.index).to_frame(name='type').reset_index()
predict_knn.to_csv(r'predicted/predict_monsters_knn.csv', index=False)

In [7]:
# Лучшие параметры для модели
gKNN_model.best_params_

{'algorithm': 'auto', 'n_neighbors': 7}

#### 2) Метод опорных векторов

In [8]:
params_for_SVC = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                 'C': [0.2, 0.4, 0.6, 0.8, 1]}
SVC_model = SVC()
gSVC_model = GridSearchCV(SVC_model, params_for_SVC, cv=5)
gSVC_model.fit(X_train, y_train)
SVC_prediction = gSVC_model.predict(X_test)

In [9]:
# Перевод предсказанных значений в серию pandas и сохранение в csv-файл
predict_svc = pd.Series(SVC_prediction, index=test.index).to_frame(name='type').reset_index()
predict_svc.to_csv(r'predicted/predict_monsters_svc.csv', index=False)

In [10]:
# Лучшие параметры для модели
gSVC_model.best_params_

{'C': 1, 'kernel': 'poly'}

#### 3) Дерево решений

In [11]:
params_for_trees = {'max_depth': range(1, 22, 2),
                    'criterion': ['entropy'],
                    }
trees_model = DecisionTreeClassifier()
gtrees_model = GridSearchCV(trees_model, params_for_trees, cv=5)
gtrees_model.fit(X_train, y_train)
trees_prediction = gtrees_model.predict(X_test) 

In [12]:
# Перевод предсказанных значений в серию pandas и сохранение в csv-файл
predict_tree = pd.Series(trees_prediction, index=test.index).to_frame(name='type').reset_index()
predict_tree.to_csv(r'predicted/predict_monsters_tree.csv', index=False)

In [13]:
# Лучшие параметры для модели
gtrees_model.best_params_

{'criterion': 'entropy', 'max_depth': 19}

#### 4) Гауссовский наивный Байес

In [14]:
gns = GaussianNB()
gns.fit(X_train, y_train)
gns_prediction = gns.predict(X_test) 

In [15]:
# Перевод предсказанных значений в серию pandas и сохранение в csv-файл
predict_gns = pd.Series(gns_prediction, index=test.index).to_frame(name='type').reset_index()
predict_gns.to_csv(r'predicted/predict_monsters_gns.csv', index=False)

#### 5) Логистическая регрессия

In [16]:
log = LogisticRegression(C=0.4)
log.fit(X_train, y_train)
log_prediction = log.predict(X_test) 

In [17]:
# Перевод предсказанных значений в серию pandas и сохранение в csv-файл
predict_log = pd.Series(log_prediction, index=test.index).to_frame(name='type').reset_index()
predict_log.to_csv(r'predicted/predict_monsters_log.csv', index=False)